In [86]:
ace_event_type_descriptions = {
    "Business:Declare-Bankruptcy": "An organization declares bankruptcy or files for legal bankruptcy protection.",
    "Business:End-Org": "An organization ceases operations, dissolves, or formally ends its existence.",
    "Business:Merge-Org": "Two or more organizations merge into a single entity.",
    "Business:Start-Org": "The establishment or founding of a new organization or company.",

    "Conflict:Attack": "A deliberate violent act intended to cause harm to people or damage to property.",
    "Conflict:Demonstrate": "A public protest, rally, or demonstration expressing opinions or demands.",

    "Contact:Meet": "A face-to-face meeting between two or more parties.",
    "Contact:Phone-Write": "Communication between parties via phone calls, letters, emails, or other written/remote means.",

    "Justice:Acquit": "A court declares a defendant not guilty of the charges.",
    "Justice:Appeal": "The act of challenging a legal decision in a higher court.",
    "Justice:Arrest-Jail": "The act of arresting or detaining a person by legal authority.",
    "Justice:Charge-Indict": "Formal accusation or indictment of a person for a crime.",
    "Justice:Convict": "A court declares a defendant guilty of a crime.",
    "Justice:Execute": "The carrying out of a death penalty sentence.",
    "Justice:Extradite": "The formal transfer of a person from one country to another for legal prosecution.",
    "Justice:Fine": "The imposition of a monetary penalty by a legal authority.",
    "Justice:Pardon": "An official act of forgiving a person for a crime and removing penalties.",
    "Justice:Release-Parole": "The release of a prisoner before the completion of their sentence under supervision.",
    "Justice:Sentence": "A court formally determines and announces the punishment for a convicted person.",
    "Justice:Sue": "A legal action in which one party files a lawsuit against another.",
    "Justice:Trial-Hearing": "A formal court proceeding or legal hearing.",

    "Life:Be-Born": "The birth of a person.",
    "Life:Die": "The death of a person.",
    "Life:Divorce": "The legal dissolution of a marriage.",
    "Life:Injure": "An event in which a person is physically harmed.",
    "Life:Marry": "The event of two people getting legally married.",

    "Movement:Transport": "The movement or transportation of people or objects from one place to another.",

    "Personnel:Elect": "The selection of a person to a position through voting.",
    "Personnel:End-Position": "A person leaving or ending a job or official position.",
    "Personnel:Nominate": "The act of formally proposing someone for a position or role.",
    "Personnel:Start-Position": "A person beginning a new job or official role.",

    "Transaction:Transfer-Money": "The transfer of money from one party to another.",
    "Transaction:Transfer-Ownership": "The transfer of ownership or control of property or assets from one party to another."
}

In [87]:
import json


gt_all = []
gt_trigger = []
with open("processed_data/ace/qwen/test.jsonl", "r") as f:
    for line in f:
        events = json.loads(json.loads(line)['response'])['events']
        e_gt = []
        triggers = []
        for trigger in events:
            # e_gt.append(str((trigger['trigger_text'], trigger['type'], [(arg['text'], arg['role']) for arg in trigger['arguments']])))
            triggers.append(str((trigger['trigger_text'], trigger['type'], ace_event_type_descriptions[trigger['type']])))
            # triggers.append(trigger['trigger_text'])
        # gt_all.append(e_gt)
        gt_trigger.append(triggers)

In [103]:
pred = []
pred_trigger = []
with open("eval_outputs/results/qwen2.5/sft_0.5B_ed/e5-bs8-lr5e-05-G2-N2-NN1/392/ace_answers.jsonl", "r") as f:
    for line in f:
        try:
            events = json.loads(json.loads(line)['text'])['events']
            e_pred = []
            triggers = []
            for trigger in events:
                # e_pred.append(str((trigger['trigger_text'], trigger['type'], [(arg['text'], arg['role']) for arg in trigger['arguments']])))
                triggers.append(str((trigger['trigger_text'], trigger['type'], trigger.get("description", ""))))
                # triggers.append(trigger['trigger_text'])
            # pred.append(e_pred)
            pred_trigger.append(triggers)
        except:
            pred_trigger.append([])

In [88]:
gt_trigger[0]

["('named', 'Personnel:Nominate', 'The act of formally proposing someone for a position or role.')"]

In [104]:
pred_trigger[0]

['(\'named\', \'Personnel:Start-Position\', "This event marks the beginning of a person\'s appointment to a new official role within an organization.")']

In [8]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("google/embeddinggemma-300m")


In [90]:
acc = 0
threshold = 0.88
all_similarities = []

for p, gt in zip(pred, gt_all):
    try:
        p_embeddings = model.encode(p)
        gt_embeddings = model.encode(gt)
        similarities = model.similarity(p_embeddings, gt_embeddings)
        all_similarities.append(similarities)
        acc += (similarities.max(-1).values > threshold).sum()
    except:
        pass

In [105]:
# print("precision: ", acc / sum(len(sublist) for sublist in pred))
# print("recall: ", acc / sum(len(sublist) for sublist in gt_all))

In [106]:
acc = 0
threshold = 0.88
all_similarities = []

for p, gt in zip(pred_trigger, gt_trigger):
    try:
        p_embeddings = model.encode(p)
        gt_embeddings = model.encode(gt)
        similarities = model.similarity(p_embeddings, gt_embeddings)
        all_similarities.append(similarities)
        acc += (similarities.max(-1).values > threshold).sum()
    except:
        pass

In [107]:
all_similarities[0]

tensor([[0.5781]])

In [108]:
acc = sum([(similarities.max(-1).values > 0.7).sum() for similarities in all_similarities])
acc

tensor(217)

In [111]:
(similarities.max(-1).values > 0.7)

tensor([False])

In [109]:
print("precision: ", acc / sum(len(sublist) for sublist in pred_trigger))
print("recall: ", acc / sum(len(sublist) for sublist in gt_trigger))

precision:  tensor(0.6977)
recall:  tensor(0.5385)


In [110]:
precision:  tensor(0.6710)
recall:  tensor(0.6476)

precision:  tensor(0.7044)
recall:  tensor(0.6799)

NameError: name 'tensor' is not defined